Autoencoders implementation

- can be used for Denoising Autoencoders

In [2]:
from tensorflow.keras.layers import Input,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
import numpy as np

encoding=32
inputimg=Input(shape=(784,))#shape of the input
encoded=Dense(encoding,activation='relu')(inputimg)#create layer then second bracket join the layer with a defined layer

decoded=Dense(784,activation='sigmoid')(encoded)#sigmoid used as values are scaled between 0 to 1 by dividing 255

autoencoder=Model(inputimg,decoded)#the whole model
encoder=Model(inputimg,encoded)#the encoder part

encodedinput=Input(shape=(encoding,))
decodedlayer=autoencoder.layers[-1]#last layer of the autoencoder model which is (784,)
decoder=Model(encodedinput,decodedlayer(encodedinput))#decodedlayer(encodedinput)joins the 2 layers together so essentially its the decoded part of the model

#separated encoder used for feature extraction and decoder can be used for data generation
autoencoder.compile(optimizer='adadelta',loss='binary_crossentropy')
#training
(xtrain,_),(xtest,_)=mnist.load_data()

#scaling the value betweemn 0 and 1
xtrain=xtrain.astype('float32')/255
xtest=xtest.astype('float32')/255

xtrain=xtrain.reshape(len(xtrain),np.prod(xtrain.shape[1:]))#data in shape(X,y,y) reshaped into (X,Y*Y)
xtest=xtest.reshape(len(xtest),np.prod(xtest.shape[1:]))#similar to above

autoencoder.fit(xtrain,xtrain,epochs=50,batch_size=256,validation_data=(xtest,xtest))

Epoch 1/50
235/235 [==============================] - 3s 5ms/step - loss: 0.6930 - val_loss: 0.6929
Epoch 2/50
235/235 [==============================] - 1s 4ms/step - loss: 0.6929 - val_loss: 0.6927
Epoch 3/50
235/235 [==============================] - 1s 4ms/step - loss: 0.6927 - val_loss: 0.6926
Epoch 4/50
235/235 [==============================] - 1s 4ms/step - loss: 0.6926 - val_loss: 0.6924
Epoch 5/50
235/235 [==============================] - 1s 4ms/step - loss: 0.6924 - val_loss: 0.6923
Epoch 6/50
235/235 [==============================] - 1s 4ms/step - loss: 0.6922 - val_loss: 0.6921
Epoch 7/50
235/235 [==============================] - 1s 3ms/step - loss: 0.6921 - val_loss: 0.6919
Epoch 8/50
235/235 [==============================] - 1s 4ms/step - loss: 0.6919 - val_loss: 0.6918
Epoch 9/50
235/235 [==============================] - 1s 4ms/step - loss: 0.6917 - val_loss: 0.6916
Epoch 10/50
235/235 [==============================] - 1s 4ms/step - loss: 0.6916 - val_loss: 0.6914

In [6]:
from keras.layers import Input,Dense,Lambda
from keras.models import Model
from tensorflow.keras import backend as K
from keras import metrics
import tensorflow as tf

ogdim=784
latentdim=2
interdim=256

x=Input(shape=(ogdim,))
h=Dense(interdim,activation='relu')(x)#needed a non linear function to stack on the layer and also avoid vanishing gradient
zmean=Dense(latentdim)(h)
zlogvar=Dense(latentdim)(h)

def sampling(args):
    zlogvar,zmean=args
    #epsilon random noise mean=0 and std=1
    epsilon=K.random_normal(shape=(K.shape(zmean)[0],latentdim),mean=0.,stddev=1.0)#random_normal creates random numbers and then the guassian noise is generated
    return zmean+K.exp(zlogvar/2)*epsilon #formula for guassian noise

z=Lambda(sampling,output_shape=(latentdim,))([zmean,zlogvar])#defines your own transformation and creates the layer instead of the default ones 
# here the custom functioning layer is sampling through which the layer is used
decoderh=Dense(interdim,activation='relu')
decodermean=Dense(ogdim,activation='sigmoid')# since output is 1 or 0 as required so sigmoid

h_decoded=decoderh(z)
x_decoded_mea=decodermean(h_decoded)

model=Model(x,x_decoded_mea)

xentloss=ogdim*metrics.binary_crossentropy(x,x_decoded_mea)#measures how close the predicted value is
kl_loss=-0.5*K.sum(1+zlogvar-K.square(zmean)-K.exp(zlogvar),axis=1)#axis =-1 makes the summation over latent dimensions as in shape(batchsize,latentdim)

vaeloss=K.mean(xentloss+kl_loss)
model.add_loss(vaeloss)
model.compile(optimizer='rmsprop')
model.summary()


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [3]:
xtrain = xtrain.reshape((-1, 28, 28, 1))
xtest = xtest.reshape((-1, 28, 28, 1))

# Normalize (optional but recommended)
xtrain = xtrain.astype('float32') / 255.
xtest = xtest.astype('float32') / 255.

In [4]:
#convolutional auto encoders
from tensorflow.keras.layers import Input,Dense,Conv2D,MaxPooling2D,UpSampling2D
from keras.models import Model

inputimg=Input(shape=(28,28,1))
x=Conv2D(16,(3,3),activation='relu',padding='same')(inputimg)#uses zeroes as padding for the digital image
x=MaxPooling2D((2,2),padding='same')(x)#reduces the size of computations and makes less sensitive to shifts in image
x=Conv2D(8,(3,3),activation='relu',padding='same')(x)
x=MaxPooling2D((2,2),padding='same')(x)
x=Conv2D(16,(3,3),padding='same',activation='relu')(x)
x=UpSampling2D((2,2))(x)#since maxpooling downsamples we need to later upsample
x=UpSampling2D((2,2))(x)
decoded=Conv2D(1,(3,3),padding='same',activation='sigmoid')(x)


autoencoder=Model(inputimg,decoded)
autoencoder.compile(optimizer='adam',loss='binary_crossentropy')
autoencoder.fit(xtrain,xtrain,epochs=5,shuffle=True,validation_data=(xtest,xtest))

Epoch 1/5
1875/1875 [==============================] - 12s 4ms/step - loss: 0.0193 - val_loss: 0.0041
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0040 - val_loss: 0.0039
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0038 - val_loss: 0.0038
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0037 - val_loss: 0.0037
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0036 - val_loss: 0.0037
